### Import packages

In [3]:
import pandas as pd

### Read in dataset into dataframe
Just need to add the dataset into the working directory

In [4]:
df = pd.read_csv("Alzheimer_s_Disease_and_Healthy_Aging_Data_20240418.csv")

In [10]:
unique_questions = df["Question"].unique()
print("Number of unique questions: %s" % len(unique_questions))
print(unique_questions)

Number of unique questions: 39
['Percentage of older adults who are experiencing frequent mental distress'
 'Percentage of older adults with a lifetime diagnosis of depression'
 'Percentage of older adults ever told they have arthritis'
 'Mean number of days with activity limitations in the past month'
 'Physically unhealthy days (mean number of days in past month)'
 'Percentage of older adults who reported influenza vaccine within the past year'
 'Fair or poor health among older adults with doctor-diagnosed arthritis'
 'Percentage of older adults getting sufficient sleep (>6 hours)'
 'Percentage of older adults who are currently obese, with a body mass index (BMI) of 30 or more'
 'Percentage of older adults who report having a disability (includes limitations related to sensory or mobility impairments or a physical, mental, or emotional condition)'
 'Percentage of older adult women who have received a mammogram within the past 2 years'
 'Percentage of at risk adults (have diabetes, as